In [2]:
from tweepy import Stream
import json
from keys import *
from companies import *
import pandas as pd
import time

In [41]:
import numpy as np
import pandas as pd
import os
import re
import string
import nltk
import matplotlib.pyplot as plt
plt.rc('figure',figsize=(17,13))
import seaborn as sns
sns.set_style('darkgrid')
import plotly.express as ex
from plotly.subplots import make_subplots
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import datetime
import warnings
warnings.filterwarnings("ignore")

In [55]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('vader_lexicon')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\darsh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\darsh\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\darsh\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [21]:
class TwitterStream(Stream):

    def __init__(self, key, secret, token, token_secret, responses, limit=50):
        """Configure the SentimentListener."""
        self.responses = responses
        self.tweet_count = 0
        self.TWEET_LIMIT = limit

        super().__init__(key, secret, token, token_secret)


    """
    Inherits from tweepy's Stream class. We are only modifying the few functions
    we need to customize, namely what to do when new tweets come through.
    """

    def get_companies(self, tweet):
        """
        :param tweet: The string text of a tweet.
        :return: A list of companies mentioned in the tweet.
        """
        tweet = tweet.lower()
        companies_mentioned = []
        for stock in stocks:
            if stock in tweet:
                companies_mentioned.append(stock)

        return companies_mentioned

    def on_data(self, raw_data):
        """
        Gets called every time a new tweet gets filtered through.
        """
        if 'extended_tweet' in self.process_data(raw_data):
            self.responses.append(self.process_data(raw_data)['extended_tweet']['full_text'])
        else:
            self.responses.append(self.process_data(raw_data)['text'])

        self.tweet_count += 1  # track number of tweets processed

        # if TWEET_LIMIT is reached, return False to terminate streaming
        if self.tweet_count == self.TWEET_LIMIT:
            self.disconnect()



    def process_data(self, raw_data):
        """
        Sends data through the Kafka log.
        """
        response = json.loads(raw_data)
        

        # Get tweet text
        if 'extended_tweet' in response:
            tweet_text = response['extended_tweet']['full_text']
        else:
            tweet_text = response['text']
        

        # Associate each company mentioned with the tweet
        companies_mentioned = self.get_companies(tweet_text)
        for corp in companies_mentioned:
            response['company_name'] = corp

        return response
            
    def on_error(self, status_code):
        """
        Returning false disconnects the stream.
        """
        if status_code == 420:
            return False

In [22]:
limit = 50
responses= []
stream = TwitterStream(API_KEY, API_KEY_SECERT,ACCESS_TOKEN, ACCESS_TOKEN_SECRET, responses, limit)

In [27]:
stream.filter(track=['Microsoft'], languages=['en'])


Stream connection closed by Twitter


In [28]:
import winsound
winsound.Beep(440, 2000)

In [29]:
print(responses)

["Microsoft's email recovery form doesn't work and asks for unobtainable verbatim information. Thousands of users have this problem, Microsoft is doing nothing about money and data loss. No longer a MS customer.Migrated everything not lost away from MS. Pursuing class action.", 'Microsoft is finally making it easier to switch default browsers in Windows 11 https://t.co/SXtEt1oRGZ via @Verge', 'RT @Wario64: Uncharted preorder is $19.99 on Microsoft Store (playable on Xbox) https://t.co/OnGlvyb1OS #ad https://t.co/mP183LHMDW', 'Microsoft will seek to reboot Activision Blizzard’s culture after $68.7-billion\xa0acquisition https://t.co/TvvyQHWD40', 'And once they have everyone hooked they will jack the price and slowly remove alternatives.', 'Oh my god. Azure data portal. I could just, not use this. I want to program not drag and drop buggy crap. Shame on you Microsoft for creating this.', 'Can’t believe Sony is putting Uncharted on Xbox!😂', 'RT @reach2ratan: Unsecured Microsoft SQL, MySQL

In [31]:
df = pd.DataFrame(responses, columns = ['Tweet'])

In [36]:
pd.set_option('display.max_colwidth', None)
df

,Tweet
0,"Microsoft's email recovery form doesn't work and asks for unobtainable verbatim information. Thousands of users have this problem, Microsoft is doing nothing about money and data loss. No longer a MS customer.Migrated everything not lost away from MS. Pursuing class action."
1,Microsoft is finally making it easier to switch default browsers in Windows 11 https://t.co/SXtEt1oRGZ via @Verge
2,RT @Wario64: Uncharted preorder is $19.99 on Microsoft Store (playable on Xbox) https://t.co/OnGlvyb1OS #ad https://t.co/mP183LHMDW
3,Microsoft will seek to reboot Activision Blizzard’s culture after $68.7-billion acquisition https://t.co/TvvyQHWD40
4,And once they have everyone hooked they will jack the price and slowly remove alternatives.
5,"Oh my god. Azure data portal. I could just, not use this. I want to program not drag and drop buggy crap. Shame on you Microsoft for creating this."
6,Can’t believe Sony is putting Uncharted on Xbox!😂
7,"RT @reach2ratan: Unsecured Microsoft SQL, MySQL servers hit by Gh0stCringe malware https://t.co/Q63rgv4jrG\n\n#CyberSecurity #hacker #infosec…"
8,RT @BondSarah_Bond: One of my favorite things about #Xbox Game Pass is how it makes gaming more accessible in more places. So awesome to br…
9,"RT @MysticRyan: And yes, Microsoft has done excellent work with their BC program. But they also monetize that program. Every game is Q&amp;A'd,…"


In [42]:
#url removal
df['Tweet'] = df['Tweet'].apply(lambda x:re.sub(r"http\S+", "", x))

In [43]:
df.head()

,Tweet
0,"Microsoft's email recovery form doesn't work and asks for unobtainable verbatim information. Thousands of users have this problem, Microsoft is doing nothing about money and data loss. No longer a MS customer.Migrated everything not lost away from MS. Pursuing class action."
1,Microsoft is finally making it easier to switch default browsers in Windows 11 via @Verge
2,RT @Wario64: Uncharted preorder is $19.99 on Microsoft Store (playable on Xbox) #ad
3,Microsoft will seek to reboot Activision Blizzard’s culture after $68.7-billion acquisition
4,And once they have everyone hooked they will jack the price and slowly remove alternatives.


In [44]:
#punctuation Removal
punctuation_removal = string.punctuation
def remove_punctuation(text):
    return text.translate(str.maketrans('', '', punctuation_removal))
df['Tweet'] = df['Tweet'].apply(lambda text: remove_punctuation(text))

In [45]:
df.head()

,Tweet
0,Microsofts email recovery form doesnt work and asks for unobtainable verbatim information Thousands of users have this problem Microsoft is doing nothing about money and data loss No longer a MS customerMigrated everything not lost away from MS Pursuing class action
1,Microsoft is finally making it easier to switch default browsers in Windows 11 via Verge
2,RT Wario64 Uncharted preorder is 1999 on Microsoft Store playable on Xbox ad
3,Microsoft will seek to reboot Activision Blizzard’s culture after 687billion acquisition
4,And once they have everyone hooked they will jack the price and slowly remove alternatives


In [46]:
#Single character and double space removal
df['Tweet'] = df['Tweet'].apply(lambda x:re.sub(r'\s+[a-zA-Z]\s+', '', x))
df['Tweet'] = df['Tweet'].apply(lambda x:re.sub(r'\s+', ' ', x, flags=re.I))

In [49]:
df.head()

,Tweet
0,Microsofts email recovery form doesnt work and asks for unobtainable verbatim information Thousands of users have this problem Microsoft is doing nothing about money and data loss No longerMS customerMigrated everything not lost away from MS Pursuing class action
1,Microsoft is finally making it easier to switch default browsers in Windows 11 via Verge
2,RT Wario64 Uncharted preorder is 1999 on Microsoft Store playable on Xbox ad
3,Microsoft will seek to reboot Activision Blizzard’s culture after 687billion acquisition
4,And once they have everyone hooked they will jack the price and slowly remove alternatives


In [51]:
#stopwords removal
STOPWORDS = set(stopwords.words('english'))
def remove_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])
df['Tweet'] = df['Tweet'].apply(lambda text: remove_stopwords(text))

In [52]:
df.head()

,Tweet
0,Microsofts email recovery form doesnt work asks unobtainable verbatim information Thousands users problem Microsoft nothing money data loss No longerMS customerMigrated everything lost away MS Pursuing class action
1,Microsoft finally making easier switch default browsers Windows 11 via Verge
2,RT Wario64 Uncharted preorder 1999 Microsoft Store playable Xbox ad
3,Microsoft seek reboot Activision Blizzard’s culture 687billion acquisition
4,And everyone hooked jack price slowly remove alternatives


In [53]:
#Emoji Removal
def remove_emoji(string):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9;"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)
df['Tweet'] = df['Tweet'].apply(str)
df['Tweet'] = df['Tweet'].apply(remove_emoji)

In [57]:
def tokenization(text):
    text = re.split('\W+', text)
    return text
df['Tokenized'] = df['Tweet'].apply(lambda x: tokenization(x.lower()))
wn = nltk.WordNetLemmatizer()
def lemmatizer(text):
    text = [wn.lemmatize(word) for word in text]
    return text
df['Lemmatized'] = df['Tokenized'].apply(lambda x: lemmatizer(x))

In [58]:
df.head()

,Tweet,Tokenized,Lemmatized
0,Microsofts email recovery form doesnt work asks unobtainable verbatim information Thousands users problem Microsoft nothing money data loss No longerMS customerMigrated everything lost away MS Pursuing class action,"[microsofts, email, recovery, form, doesnt, work, asks, unobtainable, verbatim, information, thousands, users, problem, microsoft, nothing, money, data, loss, no, longerms, customermigrated, everything, lost, away, ms, pursuing, class, action]","[microsofts, email, recovery, form, doesnt, work, asks, unobtainable, verbatim, information, thousand, user, problem, microsoft, nothing, money, data, loss, no, longerms, customermigrated, everything, lost, away, m, pursuing, class, action]"
1,Microsoft finally making easier switch default browsers Windows 11 via Verge,"[microsoft, finally, making, easier, switch, default, browsers, windows, 11, via, verge]","[microsoft, finally, making, easier, switch, default, browser, window, 11, via, verge]"
2,RT Wario64 Uncharted preorder 1999 Microsoft Store playable Xbox ad,"[rt, wario64, uncharted, preorder, 1999, microsoft, store, playable, xbox, ad]","[rt, wario64, uncharted, preorder, 1999, microsoft, store, playable, xbox, ad]"
3,Microsoft seek reboot Activision Blizzard’s culture 687billion acquisition,"[microsoft, seek, reboot, activision, blizzard, s, culture, 687billion, acquisition]","[microsoft, seek, reboot, activision, blizzard, s, culture, 687billion, acquisition]"
4,And everyone hooked jack price slowly remove alternatives,"[and, everyone, hooked, jack, price, slowly, remove, alternatives]","[and, everyone, hooked, jack, price, slowly, remove, alternative]"


In [59]:
sid = SIA()
df['sentiments'] = df["Tweet"].apply(lambda x: sid.polarity_scores(' '.join(re.findall(r'\w+',str(x).lower()))))
df['Positive Sentiment'] = df['sentiments'].apply(lambda x: x['pos']+1*(10**-6))
df['Neutral Sentiment'] = df['sentiments'].apply(lambda x: x['neu']+1*(10**-6))
df['Negative Sentiment'] = df['sentiments'].apply(lambda x: x['neg']+1*(10**-6))

In [60]:
df.head()

,Tweet,Tokenized,Lemmatized,sentiments,Positive Sentiment,Neutral Sentiment,Negative Sentiment
0,Microsofts email recovery form doesnt work asks unobtainable verbatim information Thousands users problem Microsoft nothing money data loss No longerMS customerMigrated everything lost away MS Pursuing class action,"[microsofts, email, recovery, form, doesnt, work, asks, unobtainable, verbatim, information, thousands, users, problem, microsoft, nothing, money, data, loss, no, longerms, customermigrated, everything, lost, away, ms, pursuing, class, action]","[microsofts, email, recovery, form, doesnt, work, asks, unobtainable, verbatim, information, thousand, user, problem, microsoft, nothing, money, data, loss, no, longerms, customermigrated, everything, lost, away, m, pursuing, class, action]","{'neg': 0.217, 'neu': 0.724, 'pos': 0.059, 'compound': -0.6414}",0.059001,0.724001,0.217001
1,Microsoft finally making easier switch default browsers Windows 11 via Verge,"[microsoft, finally, making, easier, switch, default, browsers, windows, 11, via, verge]","[microsoft, finally, making, easier, switch, default, browser, window, 11, via, verge]","{'neg': 0.0, 'neu': 0.781, 'pos': 0.219, 'compound': 0.4215}",0.219001,0.781001,0.000001
2,RT Wario64 Uncharted preorder 1999 Microsoft Store playable Xbox ad,"[rt, wario64, uncharted, preorder, 1999, microsoft, store, playable, xbox, ad]","[rt, wario64, uncharted, preorder, 1999, microsoft, store, playable, xbox, ad]","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}",0.000001,1.000001,0.000001
3,Microsoft seek reboot Activision Blizzard’s culture 687billion acquisition,"[microsoft, seek, reboot, activision, blizzard, s, culture, 687billion, acquisition]","[microsoft, seek, reboot, activision, blizzard, s, culture, 687billion, acquisition]","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}",0.000001,1.000001,0.000001
4,And everyone hooked jack price slowly remove alternatives,"[and, everyone, hooked, jack, price, slowly, remove, alternatives]","[and, everyone, hooked, jack, price, slowly, remove, alternative]","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}",0.000001,1.000001,0.000001
